In [ ]:
# import
import os
import uuid
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dotenv import load_dotenv
from IPython.display import display
from geopy.geocoders import Nominatim
import boto3
from IPython.display import display
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker 
from sqlalchemy import Column, Integer, String, Float 
from sqlalchemy.ext.declarative import declarative_base

Get weather data with an API

In [ ]:
French_Top_Cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre",
                "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
                "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim",
                "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
                "Gorges du Verdon", "Bormes les Mimosas", "Cassis",
                "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
                "Aigues Mortes", "Saintes Maries de la mer", "Collioure",
                "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
                "Bayonne", "La Rochelle"]

# Load variables
load_dotenv()
OPEN_WEATHER_TOKEN = os.getenv('Open_weather_token')

In [ ]:
# init dataframe
weather_df = pd.DataFrame(French_Top_Cities, columns=["cities"])
display(weather_df.sample(2))

,cities
31,Montauban
0,Mont Saint Michel


In [ ]:
# Get Cities coordinates 
def cities_to_coords(city_name) :
   
    # geocoder
    geolocator = Nominatim(user_agent="app")
    location = geolocator.geocode(city_name)

    # full_address, latitude, longitude
    full_address = location.address
    latitude = location.latitude
    longitude = location.longitude

    # return 
    return pd.Series([full_address, latitude, longitude])

## RUN
if not "full_address" in weather_df.columns :
    weather_df[["full_address", "latitude", "longitude"]] = weather_df["cities"].apply(cities_to_coords)
    display(weather_df.sample(2))

else : 
    display(weather_df.sample(2))
    print("full_address, latitude & longitude columns exists !")

,cities,full_address,latitude,longitude
31,Montauban,"Montauban, Tarn-et-Garonne, Occitanie, France ...",44.017584,1.354999
26,Saintes Maries de la mer,"Saintes-Maries-de-la-Mer, Arles, Bouches-du-Rh...",43.452277,4.428717


In [ ]:
# Get rain volume

def volume_rain(lat, lon, exclude, appid) :
    params = (
        ('lat', lat),
        ('lon', lon),
        ('exclude', exclude),
        ('appid', appid)
    )

    response = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=params)
        open_weather_7days = pd.json_normalize(response.json()["daily"])
       open_weather_7days["rain"] = open_weather_7days["rain"].apply(lambda x : 0 if np.isnan(x) else x) 

    n_days = open_weather_7days.shape[0]
    volume_rain_7days =  np.sum(open_weather_7days["rain"] * open_weather_7days["pop"]) / n_days

    return round(volume_rain_7days,3)

In [ ]:
# fill weather_df
exclude = 'current,minutely,hourly,alerts'
appid = OPEN_WEATHER_TOKEN
if not "volume_rain_7days" in weather_df.columns :
    weather_df["volume_rain_7days"] = weather_df[["latitude", "longitude"]].transpose().apply(lambda x : volume_rain(x[0], x[1], exclude, appid))

display(weather_df.sample(2))

,cities,full_address,latitude,longitude,volume_rain_7days
25,Aigues Mortes,"Aigues-Mortes, Nîmes, Gard, Occitanie, France ...",43.565823,4.191284,2.645
23,Uzes,"Uzès, Nîmes, Gard, Occitanie, France métropoli...",44.012128,4.419672,2.102


In [ ]:
# unique identifier (uuid)
if not "uuid" in weather_df.columns :
    weather_df['uuid'] = weather_df.index.to_series().map(lambda x: uuid.uuid4())

display(weather_df.sample(2))

,cities,full_address,latitude,longitude,volume_rain_7days,uuid
21,Aix en Provence,"Aix-en-Provence, Bouches-du-Rhône, Provence-Al...",43.529842,5.447474,2.566,fc388387-8378-47b6-9c60-2834649475ee
15,Grenoble,"Grenoble, Isère, Auvergne-Rhône-Alpes, France ...",45.187560,5.735782,7.462,590c8222-37f8-443a-b577-38826ab507e7


In [ ]:
# reorder columns
keep_col = ['uuid', 'cities', 'full_address', 'latitude', 'longitude', 'volume_rain_7days']
weather_df = weather_df[keep_col]

display(weather_df.sample(2))

,uuid,cities,full_address,latitude,longitude,volume_rain_7days
25,30fa5619-03b3-45aa-845e-c46de7589a71,Aigues Mortes,"Aigues-Mortes, Nîmes, Gard, Occitanie, France ...",43.565823,4.191284,2.645
27,db214571-646b-4e91-bab1-8e7475e57327,Collioure,"Collioure, Céret, Pyrénées-Orientales, Occitan...",42.525050,3.083155,1.168


In [ ]:
# Save dataframe
weather_data = "../data/weather_data.csv"
overwrite = False 

if not os.path.exists(weather_data) or overwrite==True:
    weather_df.to_csv(weather_data)

else :
    print(f"{weather_data} exists !")

In [ ]:
# fetch best destinations
top_5_destinations = weather_df.sort_values(by=['volume_rain_7days'])[:5]
top_20_destinations = weather_df.sort_values(by=['volume_rain_7days'])[:20]
display(top_5_destinations.sample(2))
display(top_20_destinations.sample(2))


,uuid,cities,full_address,latitude,longitude,volume_rain_7days
9,bb9ce686-e536-4202-ada1-9ee4db6d47d8,Chateau du Haut Koenigsbourg,"Château du Haut-Kœnigsbourg, Chemin fermé suit...",48.249490,7.344296,2.275
17,a7ad6334-307c-451f-9a55-449e22544a73,Gorges du Verdon,"Gorges du Verdon, Route des Crêtes, Les Ferrai...",43.749656,6.328562,1.450


,uuid,cities,full_address,latitude,longitude,volume_rain_7days
20,03752e13-f4e7-4ae6-ada0-9c63311a42b7,Marseille,"Marseille, Bouches-du-Rhône, Provence-Alpes-Cô...",43.296174,5.369953,2.306
23,7c8e4332-0cbf-41af-aa1b-1bec0dadbd8b,Uzes,"Uzès, Nîmes, Gard, Occitanie, France métropoli...",44.012128,4.419672,2.102


In [ ]:
# plot function
def plot(df, lat_col, lon_col, color_col, mapbox_token_file, zoom=5) :
    # read token file
    px.set_mapbox_access_token(open(mapbox_token_file).read())

    # fig
    fig = px.scatter_mapbox(df, lat=lat_col, lon=lon_col,     
                        color=color_col, size=color_col,
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=15, zoom=zoom)

    fig.show()

Scrape Booking.com

In [ ]:
# read json from scrapy spider
hotels_file = "../data/hotels_booking.json"

# file to json
hotels_json = json.load(open(hotels_file))

# json to df
hotels_df = pd.json_normalize(hotels_json)

display(hotels_df.columns)

In [ ]:
# clean some text
def clean_text(text) :
    return text.replace("\n", "")

hotels_df["name"] = hotels_df["name"].apply(lambda x : clean_text(x))
hotels_df["score"] = hotels_df["score"].apply(lambda x : clean_text(x))
hotels_df["description"] = hotels_df["description"].apply(lambda x : clean_text(x))

In [ ]:
# astring score
hotels_df["score"] = hotels_df["score"].astype(float)

In [ ]:
# save
export_hotels_csv = "../data/hotels_booking.csv"
hotels_df.to_csv(export_hotels_csv)

ETL & Datawarehouse

In [ ]:
# load dotenv variables
load_dotenv()
DBUSER = os.getenv('DBUSER')
DBPASS = os.getenv('DBPASS')
DBHOST = os.getenv('DBHOST')
DBNAME = os.getenv('DBNAME')

print(f"RDS Database : {DBNAME}")

In [ ]:
# engine : connected to our db
connection_string = f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}"
engine = create_engine(connection_string, echo=True, future=True)

In [ ]:
# sqlalchemy session to upload database
Session = sessionmaker(bind=engine)
local_session = Session()

In [ ]:
# declarative base
Base = declarative_base()

# create city table
# Let's define our table 
class City(Base):
    __tablename__ = "cities"

    uuid         = Column(String, primary_key=True)
    name         = Column(String)
    full_address = Column(String)
    latitude     = Column(Float)
    longitude    = Column(Float)

    def __repr__(self):
        return f"<City(name={self.name})>"

# weather table
class Weather(Base):
    __tablename__ = "weathers"

    weather_id    = Column(Integer, primary_key=True)
    city_uuid     = Column(String)
    volume_rain   = Column(Float)

    def __repr__(self):
        return f"<Weather(uuid={self.city_uuid}, volume={self.volume_rain})>"

# hotel table
class Hotel(Base):
    __tablename__ = "hotels"

    hotel_id     = Column(Integer, primary_key=True)
    name        = Column(String)
    city        = Column(String)
    url         = Column(String)
    image_url   = Column(String)
    score       = Column(Float)
    description = Column(String)

    def __repr__(self):
        return f"<Hotel(name={self.name}, score={self.score})>"

In [ ]:
# create_table
INIT = False
if INIT : 
    Base.metadata.create_all(engine)

Read Datalake files
weather_csv
hotels_booking_csv

In [ ]:
# s3 bucket
# dotenv variables
load_dotenv()
AWSS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

# aws session
session = boto3.Session(aws_access_key_id=AWSS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# s3 service
s3 = session.resource("s3")

# bucket 
bucket_name = 'projet_kayak'
kayak_bucket = s3.Bucket(bucket_name)

In [ ]:
# read weather data from s3
weather_key = "weather_data.csv"
weather_csv_obj = s3.Object(bucket_name, weather_key)
weather_csv = weather_csv_obj.get()['Body'].read().decode('utf-8') 

# dataframe
weather_df = pd.read_csv(weather_csv, index_col=[0])
display(weather_df.sample(2))
weather_df.columns

In [ ]:
# read hotels data from s3
hotels_key = "hotels_booking.csv"
hotels_key_obj = s3.Object(bucket_name, hotels_key)
hotels_csv = hotels_key_obj.get()['Body'].read().decode('utf-8') 

# dataframe
hotels_df = pd.read_csv(hotels_csv, index_col=[0])
display(hotels_df.sample(2))

ETL

In [ ]:
# city df
keep_col = ['uuid', 'cities', 'full_address', 'latitude', 'longitude']
city_df = weather_df[keep_col].copy()

In [ ]:
# weather df
keep_col = ['uuid', 'volume_rain_7days']
weather_df_volume = weather_df[keep_col].copy()
weather_df_volume = weather_df_volume.reset_index()
display(weather_df_volume.sample(2))

In [ ]:
# hotel dataframe
hotels_df = hotels_df.reset_index().copy()
display(hotels_df.sample(2))

DataWarehouse
Tables :

cities
weathers
hotels

In [ ]:
# fill City TABLE
for idx in range(city_df.shape[0]) : 
    # get row
    city_row = city_df.iloc[idx]

    # create city object
    city_uuid         = city_row["uuid"]
    city_name         = city_row["cities"]
    city_fulladdress  = city_row["full_address"]
    city_latitude     = city_row["latitude"]
    city_longitude    = city_row["longitude"]
    city = City(uuid=city_uuid, name=city_name, full_address=city_fulladdress,
                latitude=city_latitude, longitude=city_longitude)

    # add & commit
    # verify if cities in table
    city_in_table = local_session.query(City).filter_by(name=city_name).first()

    if not city_in_table : 
        # Add values to db 
        local_session.add(city)

        # Commit the results 
        local_session.commit()   
        
    print(city)

In [ ]:
# fill Weather TABLE
for idx in range(weather_df_volume.shape[0]) : 
    # get row
    weather_row = weather_df_volume.iloc[idx]

    # create city object
    weather_id        = int(weather_row["index"])
    city_uuid         = weather_row["uuid"]
    weather_volume    = float(weather_row["volume_rain_7days"])
    weather = Weather(weather_id=weather_id, city_uuid=city_uuid, volume_rain=weather_volume)

    # add & commit
    # verify if cities in table
    weather_in_table = local_session.query(Weather).filter_by(weather_id=weather_id).first()

    if not weather_in_table : 
        # Add values to db 
        local_session.add(weather)

        # Commit the results 
        local_session.commit()   
        
    print(weather)

In [ ]:
# fill Hotel TABLE
for idx in range(hotels_df.shape[0]) : 
    # get row
    hotel_row = hotels_df.iloc[idx]

    # create city object
    hotel_id        = int(hotel_row["index"])
    name            = hotel_row["name"]
    city            = hotel_row["city"]
    url             = hotel_row["url"]
    image_url       = hotel_row["image_url"]
    score           = hotel_row["score"]
    description     = hotel_row["description"]

    # hotel object
    hotel = Hotel(hotel_id=hotel_id, name=name, 
                    city=city, url=url, 
                    image_url=image_url, score=score, 
                    description=description)

    # add & commit
    # verify if cities in table
    hotel_in_table = local_session.query(Hotel).filter_by(name=name).first()

    if not hotel_in_table : 
        # Add values to db 
        local_session.add(hotel)

        # Commit the results 
        local_session.commit()   
        
    print(hotel)